<center><h1> Convolutional Neural Network - TensorFlow </h1><center>

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

### Load Data

In [2]:
#from tensorflow.examples.tutorials.mnist import input_data
#data = input_data.read_data_sets('data/MNIST/', one_hot=True)

In [3]:
#print(data)

#print("Size of:")
#print("- Training-set:\t\t{}".format(len(data.train.labels)))
#print("- Test-set:\t\t{}".format(len(data.test.labels)))
#print("- Validation-set:\t{}".format(len(data.validation.labels)))

### Placeholder variables

In [4]:
# Placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, 224*224], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 224, 224, 1])

# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 88], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


### Function for creating a new Convolution Layer

In [5]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights

### Function for creating a new Pooling Layer

In [6]:
def new_pool_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return layer

### Function for creating a new ReLU Layer

In [7]:
def new_relu_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

### Function for creating a new Fully connected Layer

In [8]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + biases
        
        return layer

### Create Convolutional Neural Network

In [9]:
# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=5, num_filters=6, name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=5, num_filters=12, name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Convolutional Layer 3
layer_conv3, weights_conv3 = new_conv_layer(input=layer_relu2, num_input_channels=12, filter_size=5, num_filters=24, name= "conv3")

# Pooling Layer 3
layer_pool3 = new_pool_layer(layer_conv3, name="pool3")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_pool3, name="relu3")

# Convolutional Layer 4
layer_conv4, weights_conv4 = new_conv_layer(input=layer_relu3, num_input_channels=24, filter_size=5, num_filters=48, name= "conv4")

# Pooling Layer 4
layer_pool4 = new_pool_layer(layer_conv4, name="pool4")

# RelU layer 4
layer_relu4 = new_relu_layer(layer_pool4, name="relu4")

# Convolutional Layer 5
layer_conv5, weights_conv5 = new_conv_layer(input=layer_relu4, num_input_channels=48, filter_size=5, num_filters=96, name= "conv5")

# Pooling Layer 5
layer_pool5 = new_pool_layer(layer_conv5, name="pool5")

# RelU layer 5
layer_relu5 = new_relu_layer(layer_pool5, name="relu5")

# Flatten Layer
num_features = layer_relu5.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu5, [-1, num_features])

print("Num Features:")
print(num_features)                             
                             
# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=1000, name="fc1")

# RelU layer 3
layer_relu6 = new_relu_layer(layer_fc1, name="relu6")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_relu6, num_inputs=1000, num_outputs=88, name="fc2")

Num Features:
4704


In [10]:
# Use Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

### Cost Function

In [11]:
# Use Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Optimizer

In [12]:
# Use Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

### Accuracy

In [13]:
# Accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# correct_prediction
with tf.name_scope("correct_prediction"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [15]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("Training_FileWriter/")
writer1 = tf.summary.FileWriter("Validation_FileWriter/")

In [16]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

In [17]:
num_epochs = 300
batch_size = 88
traindata_path = 'train.tfrecords'  # address to save the hdf5 file
testdata_path = 'test.tfrecords'

In [18]:
totalTrainSamples = sum(1 for _ in tf.python_io.tf_record_iterator(traindata_path))*1.0
print(totalTrainSamples)

440.0


### TensorFlow Session

In [19]:
#tf.reset_default_graph()
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    
    feature = {'train/image': tf.FixedLenFeature([], tf.string),
               'train/label': tf.FixedLenFeature([], tf.int64)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([traindata_path], num_epochs=num_epochs)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['train/image'], tf.uint8)
    # Cast label data into int32
    label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224])
    
    
    
    
    # Handle test data almost the same way
    
    featureTest =  {'test/image': tf.FixedLenFeature([], tf.string),
               'test/label': tf.FixedLenFeature([], tf.int64)}
    
    filename_queue = tf.train.string_input_producer([testdata_path], num_epochs=1)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    featuresTest = tf.parse_single_example(serialized_example, features=featureTest)
    
    image_test = tf.decode_raw(featuresTest['test/image'], tf.uint8)
    print(type(image), type(label))
    image_test = tf.reshape(image_test, [224, 224])
    label_test = tf.cast(featuresTest['test/label'], tf.int32)
    
    # Any preprocessing here ...
    
    print(type(image), type(label))
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, capacity=88, num_threads=8, min_after_dequeue=0)
    print("SIZE:", tf.size(labels))
  

    images_test, labels_test = tf.train.batch([image_test, label_test], batch_size=88, capacity=88, num_threads=8)
    
    
    

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    sess.run(init_op)
    
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Loop over number of epochs
    for epoch in range(num_epochs):

        start_time = time.time()
        train_accuracy = 0
        
        print("epoch: " + str(epoch))
    
        for batch_index in range(int(totalTrainSamples/batch_size)):
            #print("batch " + str(batch_index))
            
            img, lbl = sess.run([images, labels])
            #print(img, lbl)

            #print(img[0])
            # Get a batch of images and labels
            
            
            #print(img.shape)
            img_right_format = img.reshape((batch_size, -1))
            #print(img_right_format.shape)
            #print(result.shape)
            #pixel_lists = img.reshape(img.shape[:-3] + (-1,3))
            #result = pixel_lists[:, :, 0]
            #print(result.shape)
            
            #print(lbl.shape)
            
            # this is lbl #a = np.array([1, 0, 3])
            b = np.zeros((batch_size, 89))
            b[np.arange(batch_size), lbl] = 1
            b = np.delete(b, -1, axis=1)
            
            #print("b - shape:", b.shape)
            
            #print("b:", b)
            
            
            #print(pixel_lists.shape)
            
            x_batch = img_right_format
            y_true_batch = b
            
            #print(type(x_batch))
            #print(x_batch.shape)
            #print(type(y_true_batch))
            #print(y_true_batch.shape)
            
            # Put the batch into a dict with the proper names for placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # Calculate the accuracy on the batch of training data
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            print("Train accuracy on batch", round(train_accuracy,5))
            
            # Generate summary with the current batch of data and write to file
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, epoch*int(totalTrainSamples/batch_size) + batch_index)

            # Insert CNN CODE



            # END CNN CODE

            #print("Images " + str(img))
            #print("Labels " + str(lbl))
            #img = img.astype(np.uint8)
            #for j in range(batch_size):
            #    plt.subplot(4, 2, j+1)
            #    plt.imshow(img[j, ...])
            #    plt.title(str(lbl[j]))
            #plt.show()
            
        train_accuracy /= int(totalTrainSamples/batch_size)
        
        # Generate summary and validate the model on the entire validation set
        #summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.validation.images, y_true:data.validation.labels})
        writer1.add_summary(summ, epoch)
        
        end_time = time.time()
        
        print("Epoch "+str(epoch+1)+" completed : Time usage "+str(round(end_time-start_time, 2))+" seconds")
        print("\tAccuracy:")
        print ("\t- Training Accuracy:\t{}".format(round(train_accuracy,2)))
        #print ("\t- Validation Accuracy:\t{}".format(vali_accuracy))
        
    # Stop the threads
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    
    
    # try to get a whole test case running
    img, lbl = sess.run([images_test, labels_test])
    #print(img, lbl)
    img_right_format = img.reshape((88, -1))
    b = np.zeros((88, 89))
    b[np.arange(88), lbl] = 1
    b = np.delete(b, -1, axis=1)
    x_batch = img_right_format
    y_true_batch = b
    feed_dict_test = {x: x_batch, y_true: y_true_batch}
    test_accuracy = sess.run(accuracy, feed_dict=feed_dict_test)
    print("Test accuracy: ", test_accuracy)
    
    
    
    
    for note in range(0, 88, 5):
        # trying to get a prediction
        im = img[note].reshape((1, -1))
        feed_dict = {x: im}
        classification = sess.run(y_pred_cls, feed_dict=feed_dict)
        print("True note ", lbl[note], "Classified:", classification)   

    
    sess.close()
    


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>
SIZE: Tensor("Size:0", shape=(), dtype=int32)
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
epoch: 0
Train accuracy on batch 0.03409
Train accuracy on batch 0.09091
Train accuracy on batch 0.125
Train accuracy on batch 0.125
Train accuracy on batch 0.14773
Epoch 1 completed : Time usage 3.08 seconds
	Accuracy:
	- Training Accuracy:	0.03
epoch: 1
Train accuracy on batch 0.02273
Train accuracy on batch 0.07955
Train accuracy on batch 0.125
Train accuracy on batch 0.17045
Train accuracy on batch 0.18182
Epoch 2 completed : Time usage 1.25 seconds
	Accuracy:
	- Training Accuracy:	0.04
epoch: 2
Trai

Train accuracy on batch 0.96591
Train accuracy on batch 1.96591
Train accuracy on batch 2.96591
Train accuracy on batch 3.95455
Train accuracy on batch 4.94318
Epoch 29 completed : Time usage 1.28 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 29
Train accuracy on batch 0.96591
Train accuracy on batch 1.96591
Train accuracy on batch 2.96591
Train accuracy on batch 3.95455
Train accuracy on batch 4.94318
Epoch 30 completed : Time usage 1.28 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 30
Train accuracy on batch 0.96591
Train accuracy on batch 1.96591
Train accuracy on batch 2.96591
Train accuracy on batch 3.95455
Train accuracy on batch 4.94318
Epoch 31 completed : Time usage 1.25 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 31
Train accuracy on batch 0.96591
Train accuracy on batch 1.96591
Train accuracy on batch 2.96591
Train accuracy on batch 3.95455
Train accuracy on batch 4.94318
Epoch 32 completed : Time usage 1.28 seconds
	Accuracy:
	- Training Accuracy:	0.9

Train accuracy on batch 4.88636
Epoch 61 completed : Time usage 1.33 seconds
	Accuracy:
	- Training Accuracy:	0.98
epoch: 61
Train accuracy on batch 0.92045
Train accuracy on batch 1.85227
Train accuracy on batch 2.78409
Train accuracy on batch 3.67045
Train accuracy on batch 4.57955
Epoch 62 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.92
epoch: 62
Train accuracy on batch 0.88636
Train accuracy on batch 1.84091
Train accuracy on batch 2.72727
Train accuracy on batch 3.64773
Train accuracy on batch 4.52273
Epoch 63 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.9
epoch: 63
Train accuracy on batch 0.875
Train accuracy on batch 1.76136
Train accuracy on batch 2.73864
Train accuracy on batch 3.64773
Train accuracy on batch 4.55682
Epoch 64 completed : Time usage 1.45 seconds
	Accuracy:
	- Training Accuracy:	0.91
epoch: 64
Train accuracy on batch 0.93182
Train accuracy on batch 1.82955
Train accuracy on batch 2.75
Train accuracy on batch 3.6

Train accuracy on batch 1.88636
Train accuracy on batch 2.88636
Train accuracy on batch 3.85227
Train accuracy on batch 4.80682
Epoch 94 completed : Time usage 1.37 seconds
	Accuracy:
	- Training Accuracy:	0.96
epoch: 94
Train accuracy on batch 0.90909
Train accuracy on batch 1.82955
Train accuracy on batch 2.80682
Train accuracy on batch 3.76136
Train accuracy on batch 4.70455
Epoch 95 completed : Time usage 1.44 seconds
	Accuracy:
	- Training Accuracy:	0.94
epoch: 95
Train accuracy on batch 0.93182
Train accuracy on batch 1.84091
Train accuracy on batch 2.81818
Train accuracy on batch 3.69318
Train accuracy on batch 4.59091
Epoch 96 completed : Time usage 1.53 seconds
	Accuracy:
	- Training Accuracy:	0.92
epoch: 96
Train accuracy on batch 0.89773
Train accuracy on batch 1.86364
Train accuracy on batch 2.80682
Train accuracy on batch 3.76136
Train accuracy on batch 4.71591
Epoch 97 completed : Time usage 2.04 seconds
	Accuracy:
	- Training Accuracy:	0.94
epoch: 97
Train accuracy on ba

Train accuracy on batch 4.60227
Epoch 126 completed : Time usage 1.29 seconds
	Accuracy:
	- Training Accuracy:	0.92
epoch: 126
Train accuracy on batch 0.90909
Train accuracy on batch 1.875
Train accuracy on batch 2.85227
Train accuracy on batch 3.77273
Train accuracy on batch 4.75
Epoch 127 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.95
epoch: 127
Train accuracy on batch 0.92045
Train accuracy on batch 1.86364
Train accuracy on batch 2.81818
Train accuracy on batch 3.73864
Train accuracy on batch 4.67045
Epoch 128 completed : Time usage 1.26 seconds
	Accuracy:
	- Training Accuracy:	0.93
epoch: 128
Train accuracy on batch 0.85227
Train accuracy on batch 1.76136
Train accuracy on batch 2.70455
Train accuracy on batch 3.68182
Train accuracy on batch 4.67045
Epoch 129 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.93
epoch: 129
Train accuracy on batch 0.85227
Train accuracy on batch 1.73864
Train accuracy on batch 2.72727
Train accuracy on

Train accuracy on batch 0.90909
Train accuracy on batch 1.80682
Train accuracy on batch 2.69318
Train accuracy on batch 3.53409
Train accuracy on batch 4.42045
Epoch 159 completed : Time usage 1.26 seconds
	Accuracy:
	- Training Accuracy:	0.88
epoch: 159
Train accuracy on batch 0.88636
Train accuracy on batch 1.80682
Train accuracy on batch 2.71591
Train accuracy on batch 3.67045
Train accuracy on batch 4.55682
Epoch 160 completed : Time usage 1.26 seconds
	Accuracy:
	- Training Accuracy:	0.91
epoch: 160
Train accuracy on batch 0.875
Train accuracy on batch 1.81818
Train accuracy on batch 2.78409
Train accuracy on batch 3.77273
Train accuracy on batch 4.73864
Epoch 161 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.95
epoch: 161
Train accuracy on batch 0.89773
Train accuracy on batch 1.82955
Train accuracy on batch 2.71591
Train accuracy on batch 3.625
Train accuracy on batch 4.57955
Epoch 162 completed : Time usage 1.27 seconds
	Accuracy:
	- Training Accuracy:	0

Train accuracy on batch 3.5
Train accuracy on batch 4.43182
Epoch 191 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.89
epoch: 191
Train accuracy on batch 0.89773
Train accuracy on batch 1.77273
Train accuracy on batch 2.70455
Train accuracy on batch 3.64773
Train accuracy on batch 4.54545
Epoch 192 completed : Time usage 1.27 seconds
	Accuracy:
	- Training Accuracy:	0.91
epoch: 192
Train accuracy on batch 0.85227
Train accuracy on batch 1.71591
Train accuracy on batch 2.64773
Train accuracy on batch 3.55682
Train accuracy on batch 4.47727
Epoch 193 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.9
epoch: 193
Train accuracy on batch 0.93182
Train accuracy on batch 1.85227
Train accuracy on batch 2.78409
Train accuracy on batch 3.71591
Train accuracy on batch 4.625
Epoch 194 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.93
epoch: 194
Train accuracy on batch 0.89773
Train accuracy on batch 1.81818
Train accuracy on ba

Train accuracy on batch 0.80682
Train accuracy on batch 1.64773
Train accuracy on batch 2.48864
Train accuracy on batch 3.32955
Train accuracy on batch 4.18182
Epoch 224 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.84
epoch: 224
Train accuracy on batch 0.77273
Train accuracy on batch 1.55682
Train accuracy on batch 2.36364
Train accuracy on batch 3.21591
Train accuracy on batch 4.0
Epoch 225 completed : Time usage 1.29 seconds
	Accuracy:
	- Training Accuracy:	0.8
epoch: 225
Train accuracy on batch 0.78409
Train accuracy on batch 1.625
Train accuracy on batch 2.45455
Train accuracy on batch 3.34091
Train accuracy on batch 4.13636
Epoch 226 completed : Time usage 1.26 seconds
	Accuracy:
	- Training Accuracy:	0.83
epoch: 226
Train accuracy on batch 0.82955
Train accuracy on batch 1.67045
Train accuracy on batch 2.46591
Train accuracy on batch 3.30682
Train accuracy on batch 4.125
Epoch 227 completed : Time usage 1.29 seconds
	Accuracy:
	- Training Accuracy:	0.82
e

Train accuracy on batch 3.55682
Train accuracy on batch 4.44318
Epoch 256 completed : Time usage 1.31 seconds
	Accuracy:
	- Training Accuracy:	0.89
epoch: 256
Train accuracy on batch 0.75
Train accuracy on batch 1.60227
Train accuracy on batch 2.43182
Train accuracy on batch 3.29545
Train accuracy on batch 4.18182
Epoch 257 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.84
epoch: 257
Train accuracy on batch 0.79545
Train accuracy on batch 1.70455
Train accuracy on batch 2.54545
Train accuracy on batch 3.40909
Train accuracy on batch 4.34091
Epoch 258 completed : Time usage 1.27 seconds
	Accuracy:
	- Training Accuracy:	0.87
epoch: 258
Train accuracy on batch 0.82955
Train accuracy on batch 1.64773
Train accuracy on batch 2.57955
Train accuracy on batch 3.51136
Train accuracy on batch 4.375
Epoch 259 completed : Time usage 1.3 seconds
	Accuracy:
	- Training Accuracy:	0.88
epoch: 259
Train accuracy on batch 0.80682
Train accuracy on batch 1.67045
Train accuracy on b

Train accuracy on batch 0.79545
Train accuracy on batch 1.61364
Train accuracy on batch 2.53409
Train accuracy on batch 3.36364
Train accuracy on batch 4.25
Epoch 289 completed : Time usage 1.34 seconds
	Accuracy:
	- Training Accuracy:	0.85
epoch: 289
Train accuracy on batch 0.84091
Train accuracy on batch 1.68182
Train accuracy on batch 2.52273
Train accuracy on batch 3.39773
Train accuracy on batch 4.22727
Epoch 290 completed : Time usage 1.39 seconds
	Accuracy:
	- Training Accuracy:	0.85
epoch: 290
Train accuracy on batch 0.84091
Train accuracy on batch 1.68182
Train accuracy on batch 2.53409
Train accuracy on batch 3.375
Train accuracy on batch 4.18182
Epoch 291 completed : Time usage 1.34 seconds
	Accuracy:
	- Training Accuracy:	0.84
epoch: 291
Train accuracy on batch 0.79545
Train accuracy on batch 1.59091
Train accuracy on batch 2.38636
Train accuracy on batch 3.17045
Train accuracy on batch 3.97727
Epoch 292 completed : Time usage 1.35 seconds
	Accuracy:
	- Training Accuracy:	0

In [20]:
'''
#tf.reset_default_graph()
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    #writer.add_graph(sess.graph)
    
    print("Hello1")
    # Handle test data almost the same way
    
    featureTest =  {'test/image': tf.FixedLenFeature([], tf.string),
               'test/label': tf.FixedLenFeature([], tf.int64)}
    
    filename_queue = tf.train.string_input_producer([testdata_path], num_epochs=1)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    featuresTest = tf.parse_single_example(serialized_example, features=featureTest)
    
    image_test = tf.decode_raw(featuresTest['test/image'], tf.uint8)
    image_test = tf.reshape(image_test, [224, 224])
    label_test = tf.cast(featuresTest['test/label'], tf.int32)
    
    # Any preprocessing here ...
    
    images_test, labels_test = tf.train.batch([image_test, label_test], batch_size=88, capacity=88, num_threads=8)
    

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    sess.run(init_op)
    print("Hello1") 
    
    # try to get a whole test case running
    img, lbl = sess.run([images_test, labels_test])
    print(img, lbl)
    img_right_format = img.reshape((88, -1))
    b = np.zeros((88, 89))
    b[np.arange(88), lbl] = 1
    b = np.delete(b, -1, axis=1)
    x_batch = img_right_format
    y_true_batch = b
    feed_dict_test = {x: x_batch, y_true: y_true_batch}
    test_accuracy = sess.run(accuracy, feed_dict=feed_dict_test)
    print("Test accuracy: ", test_accuracy)
    
    
    # trying to get a prediction
    #im = img[0].reshape((1, -1))
    #print("corresponding label: ", lbl[0])
    #feed_dict = {x: im}
    #classification = sess.run(y_pred_cls, feed_dict=feed_dict)
    #print(classification)   
    #print("wrote test-dataset stuff into " + str(writer))
    
    sess.close()
    
    '''



'\n#tf.reset_default_graph()\nwith tf.Session() as sess:\n    \n    # Initialize all variables\n    sess.run(tf.global_variables_initializer())\n    \n    # Add the model graph to TensorBoard\n    #writer.add_graph(sess.graph)\n    \n    print("Hello1")\n    # Handle test data almost the same way\n    \n    featureTest =  {\'test/image\': tf.FixedLenFeature([], tf.string),\n               \'test/label\': tf.FixedLenFeature([], tf.int64)}\n    \n    filename_queue = tf.train.string_input_producer([testdata_path], num_epochs=1)\n    reader = tf.TFRecordReader()\n    _, serialized_example = reader.read(filename_queue)\n    featuresTest = tf.parse_single_example(serialized_example, features=featureTest)\n    \n    image_test = tf.decode_raw(featuresTest[\'test/image\'], tf.uint8)\n    image_test = tf.reshape(image_test, [224, 224])\n    label_test = tf.cast(featuresTest[\'test/label\'], tf.int32)\n    \n    # Any preprocessing here ...\n    \n    images_test, labels_test = tf.train.batch([